In [39]:
using Gmsh

gmsh.initialize()

gmsh.model.add("quad")

lc = 1e-2
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(1, 0, 0, lc, 2)
gmsh.model.geo.addPoint(0, 1, 0, lc, 3)
gmsh.model.geo.addPoint(1, 1, 0, lc, 4)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 4, 2)
gmsh.model.geo.addLine(4, 3, 3)
gmsh.model.geo.addLine(3, 1, 4)

gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1, true)

gmsh.model.geo.addPlaneSurface([1], 1)

gmsh.model.geo.synchronize()

gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4], 5)
gmsh.model.addPhysicalGroup(2, [1], -1, "surface")

# We can then generate a 2D mesh...
gmsh.model.mesh.setAlgorithm(2, 1, 8) # dim, surface tag, option
gmsh.model.mesh.setRecombine(2,1)
gmsh.model.mesh.generate(2)


# ... and save it to disk
gmsh.write("quad_square.msh")

# To visualize the model we can run the graphical user interface with
# `gmsh.fltk.run()'. Here we run it only if "-nopopup" is not provided in the
# command line arguments:
if !("-nopopup" in ARGS)
    gmsh.fltk.run()
end

# This should be called when you are done using the Gmsh Julia API:
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00168761s, CPU 0.001576s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Blossom: 29800 internal 398 closed
Info    : Blossom recombination completed (Wall 0.313657s, CPU 0.310435s): 10000 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.999992, min Q = 0.989606
Info    : Done meshing 2D (Wall 0.882173s, CPU 0.877851s)
Info    : 10201 nodes 10404 elements
Info    : Writing 'quad_square.msh'...
Info    : Done writing 'quad_square.msh'
-------------------------------------------------------
Version       : 4.10.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DInte

XRequest.18: BadValue 0x0


In [42]:
#..1/7: Initialize gmsh engine 
gmsh.initialize()
using WriteVTK

#..2/7: Read the mesh from file the mesh
gmsh.open("quad_square.msh")

#..3/7: Get and sort the mesh nodes
#..Observe that although the mesh is two-dimensional,
#..the z-coordinate that is equal to zero is stored as well.
#..Observe that the coordinates are stored contiguously for computational
#..efficiency
node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
nnodes = length(node_ids)
#..sort the node coordinates by ID, such that Node one sits at row 1
tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
sorted = sortslices(tosort , dims = 1);
node_ids = sorted[:,1]
xnode = sorted[:,2]
ynode = sorted[:,3]

#..4/7: Get the mesh elements
#..observe that we get all the two-dimensional triangular elements from the mesh
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
println(element_types)
for t in element_types
        name, dim, order, numv, parv, _ = gmsh.model.mesh.getElementProperties(t)
        println(" - Element type: $(name), order $(order) ($(numv) nodes in param coord: $(parv)")
end
nelements = length(element_ids[1])

#..initialize elements 
elements = [zeros(Int, 4) for i in 1:nelements];
#..loop over elements and store data into elements     
for element_id in 1:nelements
    node1_id = element_connectivity[1][4*(element_id-1)+1]
    node2_id = element_connectivity[1][4*(element_id-1)+2]
    node3_id = element_connectivity[1][4*(element_id-1)+3]
    node4_id = element_connectivity[1][4*(element_id-1)+4]
        
    # Store connectivity in a convenient format
    elements[element_id] = [node1_id, node2_id, node3_id, node4_id]
end
#println(elements)

#..5/7: Define nodes (points) and elements (cells)
#..The MeshCell data structure in provided by the WriteVTK.jl package   
points = [xnode ynode]';
cells = [MeshCell(VTKCellTypes.VTK_QUAD, el) for el in elements];

#..6/7: Create VTK file structure using nodes and elements
vtkfile  = vtk_grid("quad_mesh", points, cells);
outfiles = vtk_save(vtkfile);

#..7/7: Finalize gmsh engine 
gmsh.finalize()

Info    : Reading 'quad_square.msh'...
Info    : 9 entities
Info    : 10201 nodes
Info    : 10400 elements
Info    : Done reading 'quad_square.msh'
Int32[3]
 - Element type: Quadrilateral 4, order 1 (4 nodes in param coord: [-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0]
